In [1]:
import utils
import csv

from collections import defaultdict

import torch

from experiment import Learner

/home/km55359/.conda/envs/kmisra/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [12]:
generalization = utils.read_json("../data/experiments/single_stimuli_dative_simulation/sentences/generalization.json")

In [13]:
balanced_genset = []
for key, datives in generalization.items():
    if "do" in datives and "pp" in datives:
        for do_sent, pp_sent in zip(datives["do"], datives["pp"]):
            balanced_genset.append({"key": key, "do": do_sent, "pp": pp_sent})

In [14]:
balanced_genset

[{'key': 'nalpas',
  'do': 'danny [verb] them the cute cat .',
  'pp': 'danny [verb] the cute cat to them .'},
 {'key': 'nalpas',
  'do': 'sarah [verb] him the cute puppy .',
  'pp': 'sarah [verb] the cute puppy to him .'},
 {'key': 'nalpas',
  'do': 'dad [verb] him the cute cat .',
  'pp': 'dad [verb] the cute cat to him .'},
 {'key': 'nalpas',
  'do': 'the girl [verb] him the cute cat .',
  'pp': 'the girl [verb] the cute cat to him .'},
 {'key': 'nalpas',
  'do': 'papa [verb] us the small baby .',
  'pp': 'papa [verb] the small baby to us .'},
 {'key': 'nalpas',
  'do': 'mark [verb] him the cute baby .',
  'pp': 'mark [verb] the cute baby to him .'},
 {'key': 'nalpas',
  'do': 'danny [verb] us the small baby .',
  'pp': 'danny [verb] the small baby to us .'},
 {'key': 'nalpas',
  'do': 'jimmy [verb] them the cute puppy .',
  'pp': 'jimmy [verb] the cute puppy to them .'},
 {'key': 'nalpas',
  'do': 'sammy [verb] them the small baby .',
  'pp': 'sammy [verb] the small baby to them .'

In [15]:
with open("../data/balanced_generalization_set_three_features.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["key", "dative", "sentence"])
    for row in balanced_genset:
        writer.writerow((row['key'], "do", row['do']))
        writer.writerow((row['key'], "pp", row['pp']))

In [2]:
target = torch.load("../data/results/single_stimuli_dative_simulation_valtest_vbd_discourse_theme_given_template_1/smolm-autoreg-bpe-seed_6/raw/1_1_1_0.1_do_results/verb_embedding.pt").to("cpu")



# /home/km55359/encouraging-exposures/data/results/single_stimuli_dative_simulation_valtest_vbd_discourse_theme_given_template_1/smolm-autoreg-bpe-seed_6/raw/1_1_1_0.1_do_results/verb_embedding.pt

In [7]:
lm = Learner("kanishka/smolm-autoreg-bpe-seed_6")
lm.add_tokens()
lm.freeze_full()

In [10]:
lm.token_score("she [verb] the ball to me .")

[[('<s>', 0.0),
  ('she', -4.904932975769043),
  ('[verb]', -13.991379737854004),
  ('the', -7.50151252746582),
  ('ball', -5.1817498207092285),
  ('to', -2.408505439758301),
  ('me', -1.0440807342529297),
  ('.', -0.6565561294555664)]]

In [6]:
lm.lm.model.resize_token_embeddings().weight[lm.new_index] = target

lm.token_score("she [verb] the ball to me .")

[[('<s>', 0.0),
  ('she', -4.904930114746094),
  ('[verb]', -6.0155487060546875),
  ('the', -6.001806259155273),
  ('ball', -7.697290420532227),
  ('to', -3.8050122261047363),
  ('me', -1.605154037475586),
  ('.', -0.5210390090942383)]]